In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import kagglehub
import pandas as pd
import re

In [ ]:
path = kagglehub.dataset_download("yutkin/corpus-of-russian-news-articles-from-lenta")
file_path = f"{path}/lenta-ru-news.csv"
data = pd.read_csv(file_path)
print(data.head())

                                           url  \
0   https://lenta.ru/news/1914/09/16/hungarnn/   
1  https://lenta.ru/news/1914/09/16/lermontov/   
2  https://lenta.ru/news/1914/09/17/nesteroff/   
3   https://lenta.ru/news/1914/09/17/bulldogn/   
4       https://lenta.ru/news/1914/09/18/zver/   

                                               title  \
0  1914. Русские войска вступили в пределы Венгрии     
1  1914. Празднование столетия М.Ю. Лермонтова от...   
2                           1914. Das ist Nesteroff!   
3                    1914. Бульдог-гонец под Льежем    
4           1914. Под Люблином пойман швабский зверь   

                                                text       topic  \
0  Бои у Сопоцкина и Друскеник закончились отступ...  Библиотека   
1  Министерство народного просвещения, в виду про...  Библиотека   
2  Штабс-капитан П. Н. Нестеров на днях, увидев в...  Библиотека   
3  Фотограф-корреспондент Daily Mirror рассказыва...  Библиотека   
4  Лица, приехавшие в 

<ipython-input-2-3bd991d4562e>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [ ]:
# Заменяем NaN на пустые строки
data['text'] = data['text'].fillna("")

# Создаем словарь символов
all_text = " ".join(data['text'])
chars = sorted(list(set(all_text)))
num_chars = len(chars)

# Добавляем токен для неизвестных символов (опционально)
UNK_TOKEN = '<UNK>'
chars.append(UNK_TOKEN)
num_chars = len(chars)

char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

# Генерация данных
max_sequence_length = 20
X = []
y = []

for text in data['text'][:50]:  # Итерируемся только по 'overview'
    for i in range(len(text)):
        # Берем последовательность длиной до i+1
        end_idx = i + 1
        if end_idx > len(text):
            continue
        input_seq = text[:end_idx]
        target_char = text[end_idx] if end_idx < len(text) else UNK_TOKEN  # Обработка конца текста

        # Преобразуем символы в индексы с учетом UNK
        x_seq = [char_to_index.get(c, char_to_index[UNK_TOKEN]) for c in input_seq]
        y_char = char_to_index.get(target_char, char_to_index[UNK_TOKEN])

        X.append(x_seq)
        y.append(y_char)

# Дополнение последовательностей
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')

# One-hot encoding для y
y = to_categorical(y, num_classes=num_chars)

In [ ]:
# Создание словаря символов

data['text'] = data['text'].fillna("")
data['gen'] = data['text'][0]

# Создание словаря символов
chars = " ".join(data['gen']).split()
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

# Преобразование текста в последовательности символов
max_sequence_length = 20
X = []
y = []

for text in data['gen']:
    for i in range(len(text) - 1):
        input_seq = text[:i + 1]
        target_char = text[i + 1]
        X.append([char_to_index[char] for char in input_seq])
        y.append(char_to_index[target_char])

# Дополнение последовательностей до одинаковой длины
X = pad_sequences(X, maxlen=max_sequence_length, padding='pre')

# Преобразование целевых значений в one-hot encoding
y = to_categorical(y, num_classes=len(chars))

In [ ]:
# Параметры модели
embedding_dim = 50
lstm_units = 128

# Создание модели
model = Sequential([
    Embedding(input_dim=len(chars), output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(lstm_units),
    Dense(len(chars), activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X, y, epochs=5, batch_size=512)

Epoch 1/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 36s 201ms/step - accuracy: 0.1290 - loss: 3.7562
Epoch 2/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 29s 197ms/step - accuracy: 0.2023 - loss: 2.9921
Epoch 3/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 28s 191ms/step - accuracy: 0.2242 - loss: 2.8202
Epoch 4/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 40s 188ms/step - accuracy: 0.2380 - loss: 2.7320
Epoch 5/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 42s 194ms/step - accuracy: 0.2434 - loss: 2.6787


In [ ]:
# Функция для генерации текста
def generate_text(seed_text, num_chars_to_generate):
    for _ in range(num_chars_to_generate):
        # Преобразование seed_text в последовательность индексов
        input_seq = [char_to_index[char] for char in seed_text]
        input_seq = pad_sequences([input_seq], maxlen=max_sequence_length, padding='pre')

        # Предсказание следующего символа
        predicted_probs = model.predict(input_seq, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        predicted_char = index_to_char[predicted_index]

        # Добавление предсказанного символа к seed_text
        seed_text += predicted_char

    return seed_text

# Генерация текста
seed_texts = ["Бои", "лица", "дни"]
for seed in seed_texts:
    generated_text = generate_text(seed, num_chars_to_generate=50)
    print(f"Начальная фраза: '{seed}'")
    print(f"Сгенерированный текст: {generated_text}")
    print("-" * 50)

Начальная фраза: 'Бои'
Сгенерированный текст: Бои пренито пренито пренито пренито пренито пренито п
--------------------------------------------------
Начальная фраза: 'лица'
Сгенерированный текст: лица пренито пренито пренито пренито пренито пренито п
--------------------------------------------------
Начальная фраза: 'дни'
Сгенерированный текст: дни пренито пренито пренито пренито пренито пренито п
--------------------------------------------------
